# Prompt Engineering: Use OpenAI to Analyze Twitter Data 
This is a simple tutorial teaching prompt engineering basics and analyzing Twitter data with OpenAI large language models (LLM).
Please purchase an [OpenAI API](https://openai.com/index/openai-api/) and store it in a safe place. This tutorial uses [AWS Secretes Manager](https://aws.amazon.com/secrets-manager/) to store the API keys.  

## Large Language Model Basics
LLM repeatable predicts the next world using supervised learning. To predict the following sentence: 

`Learning data science in the cloud with AI`

A model needs to learn to predict the following steps:

|Input|Output|
|:---|---|
|Learning data science |in |
|Learning data science in |the | 
|Learning data science in the |cloud |
|Learning data science in the cloud |with |
|Learning data science in the cloud with |AI|

To train an LLM model:
1. Training a base LLM model on a large amount of training data to predict the next word 
2. Fine-tune on examples where outputs follow instructions in the input 
3. Human rates quality of different LLM outputs 
4. Tune LLM to generate outputs with higher rates using RLHF (Reinforcement learning from human feedback)

## Set up OpenAI Models

Load the API keys with AWS Secrets Manage Function 

In [1]:
import boto3
from botocore.exceptions import ClientError
import json

def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    return json.loads(secret)

## Install Python libraries.

- pymongo: manage the MongoDB database
- openai: call the OpenAI APIs.

In [2]:
pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.1/599.1 kB 35.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 43.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


Load the OpenAI API key and define a `openai_help` function.

In [4]:
from openai import OpenAI

openai_api_key  = get_secret('openai')['api_key']
client = OpenAI(api_key=openai_api_key)
model = 'gpt-4o'
temperature = 0

def openai_help(messages, model=model, temperature =temperature ):
    messages = messages
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature

    )
    return response.choices[0].message.content

Temperature: 
- Low temperature: always choose the most likely response, reliable, predictable responses  
- High temperature: diverse responses, more creative responses

Tokens and Models: 
- LLM predicts tokens, which are commonly occurring sequences of characters. 
- One token is about four characters in English, and 100 tokens are roughly 75 words. Check [token estimate](https://platform.openai.com/tokenizer).
- Different models can process various amounts of tokens at different performance levels and costs. Check [OpenAI models](https://platform.openai.com/docs/models) for more details.

Roles:
- system: specify the overall tone or behavior of the assistant 
- user: instruction given to the LLM
- assistant: LLM responded content, we also can provide content in few-shot promoting or histories of conversations


A simple example using [gtp-4o](https://platform.openai.com/docs/models/gpt-4o) and temperature 0.

In [5]:
messages = [{"role": "user", "content": "What is the capital of USA"}]

print(openai_help(messages))

The capital of the United States is Washington, D.C.


Add a system message asking LLM to act as a high school teacher with different temperatures.

In [6]:
messages = [
    {"role": "system", "content": "use tone as a high school teacher"},
    {"role": "user", "content": "What is the capital of USA"}
    ]

print(openai_help(messages, temperature = 0.8))

The capital of the United States is Washington, D.C.


Add assistant messages to teach LLM what `##` is.

In [7]:
messages = [
    {"role": "user", "content": "What is 1##1"},
    {"role": "assistant", "content": "it is 11"},
    {"role": "user", "content": "What is 2##2"},
    {"role": "assistant", "content": "it is 22"},
    {"role": "user", "content": "What is 3##3"},
    ]
print(openai_help(messages))

It is 33.


## Prompt Engineering Principles 
- Use delimiters to separate different parts of a prompt to provide clear instructions and prevent prompt injections.
- Structure outputs in JSON documents or other formats to use the outputs in subsequent steps 
- Few-shot promoting: provide successful examples of a task and then ask the model to perform a similar task. 
- Chain of thought reasoning: request a series of reasoning steps in prompts to help the model achieve correct answers
- Chain of prompts: split a task into multiple prompts where each prompt can focus on a sub-task at a time and take different actions at different stages. It saves tokens, is easier to test, can involve human input, or use external tools.
- Interactive process 
  1. Try something first 
  2. Analyses the result, identify errors, and redefine the prompt 
  3. Test the prompts with different datasets 


An example using delimiters, structured output and few-shot promoting:

In [8]:
delimiter = '###'
sentence1 = 'I love cat.'
sentence2 = 'I love dog.'
messages = [
    {"role": "system", "content": f"""analyze the sentiment in a sentence delimitered by {delimiter},
                                     return the result as a JSON document"""},
    {"role": "user", "content": f"{delimiter}{sentence1}{delimiter}"},
    {"role": "assistant", "content": "{sentiment:positive}"},
    {"role": "user", "content": f"{delimiter}{sentence2}{delimiter}"}
    ]

print(openai_help(messages))

{ "sentiment": "positive" }


## Analyze Twitter data

### Connect to the MongoDB cluster

In [9]:
import pymongo
from pymongo import MongoClient
mongodb_connect = get_secret('mongodb')['connection_string']

mongo_client = MongoClient(mongodb_connect)
db = mongo_client.demo # use or create a database named demo
tweet_collection = db.tweet_collection #use or create a collection named tweet_collection
tweet_collection.create_index([("tweet.id", pymongo.ASCENDING)],unique = True) # make sure the collected tweets are unique

'tweet.id_1'

### Extract Tweets

In [10]:
filter={

    
}
project={
    'tweet.text': 1, 
    'tweet.id': 1
}
#rename the client to mongo_client
result = mongo_client['demo']['tweet_collection'].find(
  filter=filter,
  projection=project
)

In [11]:
tweet_data = []
for tweet in result:
    tweet_data.append(tweet['tweet']['text'])

In [12]:
print('Number of tweets: ',len(tweet_data))

Number of tweets:  200


### Summarization 
- Analyze election tweets with delimiters 
- Change the size of the summarization 
- Summarize tweets and focus on different perspectives. 

In [13]:
messages = [
    {"role": "system", "content": f"""provide a brief summary of the tweets delimited by {delimiter}"""},
    {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter}"},
    ]

print(openai_help(messages))

The tweets discuss various aspects of the upcoming elections, with a focus on the 2024 U.S. presidential election. Key themes include concerns about election integrity, voter ID laws, and the influence of dark money. There are mentions of Donald Trump's candidacy, his chances of winning, and allegations of election interference. Some tweets express skepticism about media coverage and polling accuracy, while others highlight voter enthusiasm and early voting trends. Additionally, there are references to international politics, such as Israel's potential actions before the U.S. election, and discussions about political figures like Kamala Harris and Joe Biden. The tweets also touch on local elections and political dynamics in states like Georgia and Pennsylvania.


In [14]:
messages = [
    {"role": "system", "content": f"""provide a brief summary of the tweets delimited by {delimiter},
                                    limit the summary to 20 words"""},
    {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter}"},
    ]

print(openai_help(messages))

The tweets discuss election-related topics, including voter turnout, election integrity, political predictions, and concerns about misinformation and interference.


In [15]:
messages = [
    {"role": "system", "content": f"""provide a brief summary of the tweets delimited by {delimiter},
                                    focus on how people discuss AI,
                                    limit the summary to 50 words"""},
    {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter}"},
    ]

print(openai_help(messages))

The tweets reflect a polarized discussion on AI's role in elections, with concerns about misinformation, election integrity, and potential interference. Some express skepticism about AI's influence on media narratives and election outcomes, while others highlight the need for transparency and security in voting systems.


### Moderation 
- Iterate each tweet and use the [moeration endpoint](https://platform.openai.com/docs/api-reference/moderations) to identify flagged tweets
- Print flagged tweets


In [16]:
def flag_help(tweet):
    response = client.moderations.create(
        model="omni-moderation-latest",
        input=tweet)

    if response.results[0].flagged:
        print('===')
        cat_dict = response.results[0].categories.to_dict()
        for cat in cat_dict.keys():
            if cat_dict.get(cat):
                print (cat)
                print(tweet)

In [17]:
for tweet in tweet_data:
    flag_help(tweet)

===
harassment
@LauraLoomer Their time will be coming to an end soon once Trump wins this election. They will be running and hiding. No more protection from this treasonous administration.
===
harassment
@DC_Draino Kamala let in a bunch of terrorist! DHS caught one planning an attack on election day! He was in oklahoma of all places! They are everywhere!
===
harassment
The media and the left spend all year claiming crime is down. They “fact check” Trump at the debate on it. Weeks before the election, the FBI quietly “updates” the data to show that crime is in fact up. The leftist response is to say “relax, rightoid, the FBI does this REGULARLY”
===
harassment
RT @maddenifico: Some of you self-righteous motherfuckers on the left are once again overthinking this. It's as if you faux-prog frauds lea…
===
harassment
@BretBaier This will be the biggest pillow fight of the election process. She was indeed the Border Czar, she lied to us about the health of President Biden, used tax payer $ t

### Transforming
- Translating to a different language 
- Transform tones, such as formal vs. informal.  


In [18]:
for tweet in tweet_data:
    messages = [
        {"role": "system", "content": f"""translate the tweets delimited by {delimiter} into Chinese"""},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]

    print(openai_help(messages).strip(delimiter))

阅读底部的字幕

我祈祷他也能活到参加2028年的选举
这是一个拥有坚定决心的虔诚信徒 https://t.co/WvI9ewX6n1
@MarioNawfal 是时候让更多的声音在即将到来的选举中被听到了，而不是那些已经去世却仍然能够投票的人！
@joshtheobald3 我们必须这样做，妻子在选举日工作，她下班太晚，来不及投票，我们将在21日投票。
@LauraLoomer 一旦特朗普赢得这次选举，他们的时代就要结束了。他们将会逃跑和躲藏。这个叛国政府将不再受到保护。
重要信息：来自@SimonWDC的Hopium Chronicles子刊今天的内容：“要注意2024年红色浪潮努力的规模。它远比2022年大，并且包括像Polymarket和Elon这样的新角色。他们正在努力制造一种选举正在失去的印象…… https://t.co/7At7SvhOaM
在他们的选举报道节目中，India Today 将埃克纳特·辛德描绘为马拉塔士兵，而乌达夫·萨克雷则被描绘为莫卧儿士兵。

也许这是一个弗洛伊德式的失误，或者可能是故意的，但似乎 India Today 这次准确地把握了基层的情绪。https://t.co/LiXvIYzSYI
RT @PATRICKHENRY97: 那么，为什么自由派不希望投票需要身份证？哦，我知道他们再也不会赢得选举。身份证和一天投票。L…
我妈妈在2016年选举日时无法投票，2020年因为疫情，他们没有派特别选举官员到她所在的设施，所以这次是她第一次有机会投票反对特朗普，她对此感到非常兴奋。
RT @OccupyDemocrats: 突发新闻：唐纳德·特朗普在荒谬地声称权力和平移交后，被采访者羞辱…
RT @mtgreenee: 乔治亚州的女性们充满激情，准备投票！

她们担心孩子们的未来和家庭的安全。

预…
RT @SimonWDC: 昨晚特朗普在镜头前迷失了方向，整整39分钟。观看视频，令人震惊。

他的执政能力，…
转发 @FrankLuntz: 来自 @NPR 的报道：“唐纳德·特朗普似乎再次在这次总统选举中处于主导地位。
@CentristMadness 因为没有其他人了解它们的运作方式或真正关心

这就是我们应得的选举和媒体环境
RT @shaun_vids: 选举之后，他们可以在不实际阻止内塔尼亚胡的情况下，宣称对他强硬。真

In [20]:
for tweet in tweet_data:
    messages = [
        {"role": "system", "content": f"""rewrite the tweets delimited by {delimiter} in the tone like Stewie """},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]

    print(openai_help(messages).strip(delimiter))

Ah, the ever-informative chyron, always there to enlighten the masses. 

I do hope he remains with us to cast his vote in the 2028 election as well. Such a paragon of divine fortitude, isn't he? https://t.co/WvI9ewX6n1
Ah, yes, the election season is upon us, and it seems even the dearly departed are exercising their right to vote. How delightfully democratic! Perhaps we should start campaigning in cemeteries.
Ah, the trials and tribulations of adult responsibilities. It seems the missus is otherwise engaged on the grand day of democracy, so you'll be casting your votes on the 21st. How delightfully proactive of you.
Oh, Laura, my dear, how delightfully naive. Once the Trumpster claims his throne, those scoundrels will be scurrying like the vermin they are, seeking refuge from their own treacherous deeds. No more coddling from this dastardly regime, I assure you.
IMPORTANT: From @SimonWDC's Hopium Chronicles substack today: "Do pay attention, you simpletons, to the grandiose scale of t

KeyboardInterrupt: 

### Inferring
- Use step-by-step instructions with delimiters to:
  1. Identify sentiments
  2. Identify emotions
  3. Extract mentioned people's names
  3. Identify whether a tweet supports Democratic, Republican, or unknown 
  4. Extract outputs into a structured JSON document. 
- Identify topics from Tweets. 


In [21]:
for tweet in tweet_data:
    messages = [
        {"role": "system", "content": f"""analyze the tweet delimited by {delimiter} in the following steps:
                                        step 1 {delimiter} identify the tweet sentiment in a single word, either positive, negative or neutral;
                                        step 2 {delimiter} identify the emotions expressed in the tweet with a single word;
                                        step 3 {delimiter} extract the mentioned peoples;
                                        step 4 {delimiter} detect whether the tweet support Democratic or Replublican, return the resunt in a single word;
                                        step 5 {delimiter} organize the result in a json document with the keys <sentiment>, <emontion>,<mentioned>, <support>
                                         Do not wrap the json codes in JSON markers and only return the json document"""},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]
    print(openai_help(messages))

{
  "sentiment": "positive",
  "emotion": "hopeful",
  "mentioned": [],
  "support": "neutral"
}
{
  "sentiment": "negative",
  "emotion": "sarcasm",
  "mentioned": ["MarioNawfal"],
  "support": "Republican"
}
{
  "sentiment": "neutral",
  "emotion": "determination",
  "mentioned": ["@joshtheobald3"],
  "support": "neutral"
}
{
  "sentiment": "negative",
  "emotion": "anger",
  "mentioned": ["LauraLoomer", "Trump"],
  "support": "Republican"
}
{
  "sentiment": "neutral",
  "emotion": "awareness",
  "mentioned": ["SimonWDC", "Polymarket", "Elon"],
  "support": "Democratic"
}
{
  "sentiment": "neutral",
  "emotion": "curiosity",
  "mentioned": ["Eknath Shinde", "Uddhav Thackeray", "India Today"],
  "support": "neutral"
}
{
  "sentiment": "negative",
  "emotion": "skepticism",
  "mentioned": ["PATRICKHENRY97"],
  "support": "Republican"
}
{
  "sentiment": "positive",
  "emotion": "excitement",
  "mentioned": ["Trump"],
  "support": "Democratic"
}
{
  "sentiment": "negative",
  "emotion": 

KeyboardInterrupt: 

In [22]:

messages = [
        {"role": "system", "content": f"""analyze the tweet delimited by {delimiter} to identify 10 topics, 
                                  Do not wrap the json codes in JSON markers """},
        {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter} "}]
print(openai_help(messages))

{
  "topics": [
    "Election Integrity and Fraud Allegations",
    "Donald Trump's Election Prospects",
    "Kamala Harris's Campaign and Strategy",
    "Voter ID Laws and Voting Accessibility",
    "Media Influence and Election Coverage",
    "Polling and Election Predictions",
    "Election Day Logistics and Early Voting",
    "Political Campaigns and Strategies",
    "International Influence and Election Security",
    "Public Sentiment and Political Polarization"
  ]
}


### Expanding with multiple prompts 
- Identify which party receives majority supports
- Provide contexts in the system message
- Create a chatbot to answer users’ inquiry  


In [23]:
analysis_result = []
from tqdm import tqdm
for tweet in tqdm(tweet_data):
    messages = [
        {"role": "system", "content": f"""analyze the tweet delimited by {delimiter} in the following steps:
                                        step 1 {delimiter} identify the tweet sentiment in a single word, either positive, negative or neutral;
                                        step 2 {delimiter} identify the emotions expressed in the tweet with a single word;
                                        step 3 {delimiter} extract the mentioned peoples;
                                        step 4 {delimiter} detect whether the tweet support Democratic or Replublican, return the resunt in a singple word;
                                        step 5 {delimiter} organize the result in a json document with the keys <sentiment>, <emontion>,<mentioned>, <support>
                                         Do not wrap the json codes in JSON markers and only return the json document"""},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]
    analysis_result.append(openai_help(messages))


100%|██████████| 200/200 [03:11<00:00,  1.05it/s]


In [24]:
print(analysis_result)

['{\n  "sentiment": "positive",\n  "emotion": "hopeful",\n  "mentioned": [],\n  "support": "neutral"\n}', '{\n  "sentiment": "negative",\n  "emotion": "sarcasm",\n  "mentioned": ["MarioNawfal"],\n  "support": "Republican"\n}', '{\n  "sentiment": "neutral",\n  "emotion": "determination",\n  "mentioned": ["@joshtheobald3"],\n  "support": "neutral"\n}', '{\n  "sentiment": "negative",\n  "emotion": "anger",\n  "mentioned": ["LauraLoomer", "Trump"],\n  "support": "Republican"\n}', '{\n  "sentiment": "neutral",\n  "emotion": "awareness",\n  "mentioned": ["SimonWDC", "Polymarket", "Elon"],\n  "support": "Republican"\n}', '{\n  "sentiment": "neutral",\n  "emotion": "curiosity",\n  "mentioned": ["Eknath Shinde", "Uddhav Thackeray", "India Today"],\n  "support": "neutral"\n}', '{\n  "sentiment": "negative",\n  "emotion": "skepticism",\n  "mentioned": ["PATRICKHENRY97"],\n  "support": "Republican"\n}', '{\n  "sentiment": "positive",\n  "emotion": "excitement",\n  "mentioned": ["Trump"],\n  "suppo

In [25]:
messages = [
        {"role": "system", "content": f"""analyze the tweet analysis reuslt delimited by {delimiter} in the following steps:
                                        step 1 {delimiter} count the number of tweets that support Democratic and Republican;
                                        step 2 {delimiter} identify the common sentiments and emotoions to each mentioned people;
                                        step 3 {delimiter} organize the result in a json document with keys <Democratic count>, <Republican count>, <people name>
                                         Do not wrap the json codes in JSON markers and only return the json document"""},
        {"role": "user", "content": f"{delimiter}{analysis_result}{delimiter} "}]
analysis_summary = openai_help(messages)
print(analysis_summary)

{
  "Democratic count": 22,
  "Republican count": 54,
  "people name": {
    "Democratic": {
      "common sentiments": ["negative", "neutral", "positive"],
      "common emotions": ["frustration", "concern", "admiration", "supportive", "informative"]
    },
    "Republican": {
      "common sentiments": ["negative", "neutral", "positive"],
      "common emotions": ["frustration", "anger", "informative", "anticipation", "skepticism"]
    }
  }
}


## Create a chatbot

In [26]:
from openai import OpenAI

openai_api_key  = get_secret('openai')['api_key']
client = OpenAI(api_key=openai_api_key)
model = 'gpt-4o'
temperature = 0

chat_history = [

{"role": "system", "content": f"""you are a chabot answer user questions based on the tweets,
                                {delimiter}{tweet_data}{delimiter}, 
                                if user mentioned a people name in the {delimiter}{analysis_summary}{delimiter} people field,report the corresponding sentiment and emotion,
                            
                            """}
]

def chatbot(prompt):

    chat_history.append({"role": "user", "content": prompt})

    response = client.chat.completions.create(
        model=model,  # Use the model you prefer
        messages=chat_history
    )

    reply = response.choices[0].message.content

    chat_history.append({"role": "assistant", "content": reply})
    
    return reply

In [28]:
while True:
    user_input = input("You: ")
    if user_input.lower() in ['exit', 'quit']:
        print("Chatbot: Goodbye!")
        break
    reply = chatbot(user_input)
    print(f"Chatbot: {reply}")

You:  what do they talk about?


Chatbot: The tweets discuss a variety of topics centered around elections, primarily focusing on:

1. **Election Integrity and Interference:** Concerns regarding election interference, fraud, and the integrity of the electoral process. There are mentions of court rulings, media influence, and accusations against political figures and parties.

2. **Political Support and Predictions:** Expressions of support for particular candidates, such as Donald Trump and Kamala Harris, along with predictions about election outcomes and the importance of certain upcoming elections.

3. **Voter Engagement:** Encouragement for voter turnout, discussing the significance of voting and the impact of specific voter groups on the elections.

4. **Media Influence:** Criticism and discussion about how the media reports on elections and political figures, including accusations of bias and misinformation.

5. **Specific Incidents and Rulings:** References to specific events, such as legal decisions in various 

You:  quit


Chatbot: Goodbye!


## Reference
- Isa Fulford and Andrew Ng. n.d.-a. *“Building Systems with the ChatGPT API.”* DeepLearning.AI. Accessed October 25, 2024. https://www.deeplearning.ai/short-courses/building-systems-with-chatgpt/.
- ———. n.d.-b. *“ChatGPT Prompt Engineering for Developers.”* DeepLearning.AI. Accessed October 25, 2024. https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/.
- OpenAI. n.d. *“OpenAI Documents.”* OpenAI. Accessed October 18, 2024. https://platform.openai.com.
